In [1]:
# Import required libraries and dependencies

In [2]:
#import keras as krs
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=4, suppress=True)
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline
from pathlib import Path

In [3]:
def assess_NA(data):
    """
    Returns a pandas dataframe denoting the total number of NA values and the percentage of NA values in each column.
    The column names are noted on the index.
    
    Parameters
    ----------
    data: dataframe
    """
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; omit if you want to keep all rows
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

# us_balance_annual

In [4]:
us_balance_annual = pd.read_csv(
    Path('us-balance-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';',

)


us_balance_annual.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,A,45846,USD,2008,Q4,2008-10-31,2008-12-19,2009-12-21,363000000.0,371000000.0,...,NaN,1.330000e+09,2.125000e+09,3.118000e+09,4448000000,7.416000e+09,-7.470000e+09,2.791000e+09,2.559000e+09,7007000000
1,A,45846,USD,2009,Q4,2009-10-31,2009-12-21,2010-12-20,346000000.0,346000000.0,...,1.000000e+06,1.123000e+09,2.904000e+09,3.975000e+09,5098000000,7.558000e+09,-7.627000e+09,2.760000e+09,2.514000e+09,7612000000
2,A,45846,USD,2010,Q4,2010-10-31,2010-12-20,2011-12-16,347000000.0,353000000.0,...,1.501000e+09,3.083000e+09,2.190000e+09,3.377000e+09,6460000000,7.910000e+09,-8.038000e+09,3.444000e+09,3.236000e+09,9696000000
3,A,45846,USD,2011,Q4,2011-10-31,2011-12-16,2012-12-20,347000000.0,355000000.0,...,2.530000e+08,1.837000e+09,1.932000e+09,2.904000e+09,4741000000,8.271000e+09,-8.535000e+09,4.456000e+09,4.316000e+09,9057000000
4,A,45846,USD,2012,Q4,2012-10-31,2012-12-20,2013-12-19,348000000.0,353000000.0,...,2.500000e+08,1.893000e+09,2.112000e+09,3.458000e+09,5351000000,8.495000e+09,-8.707000e+09,5.505000e+09,5.185000e+09,10536000000


In [5]:
supa_balance = us_balance_annual.drop(['SimFinId', 'Currency'], axis=1)


In [6]:
balance_data = supa_balance.groupby(['Ticker','Fiscal Year']).mean()
balance_data

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                            1.429000e+09   
       2009                                            2.493000e+09   
       2010                                            2.649000e+09   
       2011                                            3.527000e+09   
       2012                                            2.351000e+09   
...                                                             ...   
ZYXI   2015                                            8.000000e+03   
       2016                                            2.470000e+05   
       2017                                            5.565000e+06   
       2018                                            1.012800e+07   
       2019                                            1.404000e+07   

                    Accounts & Notes Receivable   Inventories  \
Ticker Fiscal Year                                              
A      2008                         770000000.0  6.460000e+08   
       2009                         595000000.0  5.520000e+08   
       2010                         869000000.0  7.160000e+08   
       2011                         860000000.0  8.980000e+08   
       2012                         923000000.0  1.014000e+09   
...                                         ...           ...   
ZYXI   2015                           2426000.0  3.050000e+05   
       2016                           3028000.0  1.070000e+05   
       2017                           2185000.0  4.230000e+05   
       2018                           2791000.0  8.370000e+05   
       2019                           5833000.0  2.378000e+06   

                    Total Current Assets  Property, Plant & Equipment, Net  \
Ticker Fiscal Year                                                           
A      2008                 3.182000e+09                      8.240000e+08   
       2009                 3.961000e+09                      8.450000e+08   
       2010                 6.169000e+09                      9.800000e+08   
       2011                 5.569000e+09                      1.006000e+09   
       2012                 4.629000e+09                      1.164000e+09   
...                                  ...                               ...   
ZYXI   2015                 2.766000e+06                      8.010000e+05   
       2016                 3.422000e+06                      5.800000e+05   
       2017                 8.371000e+06                      1.880000e+05   
       2018                 1.432400e+07                      8.190000e+05   
       2019                 2.256600e+07                      8.580000e+05   

                    Long Term Investments & Receivables  \
Ticker Fiscal Year                                        
A      2008                                 206000000.0   
       2009                                 163000000.0   
       2010                                 142000000.0   
       2011                                 117000000.0   
       2012                                 109000000.0   
...                                                 ...   
ZYXI   2015                       

In [38]:
balance_data_cash = balance_data[["Cash, Cash Equivalents & Short Term Investments", 
                                  "Accounts & Notes Receivable", 
                                  'Treasury Stock', 
                                  'Retained Earnings']]

balance_sheet_cash_equal = balance_data_cash.dropna().astype(int)
balance_sheet_cash_equal

Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                              1429000000   
       2009                                             -2147483648   
       2010                                             -2147483648   
       2011                                             -2147483648   
       2012                                             -2147483648   
...                                                             ...   
ZTS    2019                                              1934000000   
ZYXI   2016                                                  247000   
       2017                                                 5565000   
       2018                                                10128000   
       2019                                                14040000   

                    Accounts & Notes Receivable  Treasury Stock  \
Ticker Fiscal Year                                                
A      2008                           770000000     -2147483648   
       2009                           595000000     -2147483648   
       2010                           869000000     -2147483648   
       2011                           860000000     -2147483648   
       2012                           923000000     -2147483648   
...                                         ...             ...   
ZTS    2019                          1086000000     -2042000000   
ZYXI   2016                             3028000               0   
       2017                             2185000         -243000   
       2018                             2791000        -3675000   
       2019                             5833000        -3846000   

                    Retained Earnings  
Ticker Fiscal Year                     
A      2008               -2147483648  
       2009               -2147483648  
       2010               -2147483648  
       2011               -2147483648  
       2012               -2147483648  
...                               ...  
ZTS    2019               -2147483648  
ZYXI   2016                  -9776000  
       2017                  -2411000  
       2018                   4864000  
       2019                  14356000  

[8612 rows x 4 columns]

In [41]:
balance_sheet_cash_equal.reset_index(inplace=True)

In [91]:
bal_sheey_cash_set = balance_sheet_cash_equal.set_index(['Ticker'])

In [92]:
bal_sheey_cash_set

,Fiscal Year,"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Treasury Stock,Retained Earnings
Ticker,,,,,
A,2008,1429000000,770000000,-2147483648,-2147483648
A,2009,-2147483648,595000000,-2147483648,-2147483648
A,2010,-2147483648,869000000,-2147483648,-2147483648
A,2011,-2147483648,860000000,-2147483648,-2147483648
A,2012,-2147483648,923000000,-2147483648,-2147483648
...,...,...,...,...,...
ZTS,2019,1934000000,1086000000,-2042000000,-2147483648
ZYXI,2016,247000,3028000,0,-9776000
ZYXI,2017,5565000,2185000,-243000,-2411000


In [95]:
balance_sheet_tanglibles = balance_data[["Property, Plant & Equipment, Net",
                           "Retained Earnings", 
                           "Total Equity"]]

balance_sheet_tanglibles_gold = balance_sheet_tanglibles.dropna().astype(int)
balance_sheet_tanglibles_gold

Property, Plant & Equipment, Net  Retained Earnings  \
Ticker Fiscal Year                                                        
A      2008                                824000000        -2147483648   
       2009                                845000000        -2147483648   
       2010                                980000000        -2147483648   
       2011                               1006000000        -2147483648   
       2012                               1164000000        -2147483648   
...                                              ...                ...   
ZYXI   2015                                   801000           -9845000   
       2016                                   580000           -9776000   
       2017                                   188000           -2411000   
       2018                                   819000            4864000   
       2019                                   858000           14356000   

                    Total Equity  
Ticker Fiscal Year                
A      2008          -2147483648  
       2009          -2147483648  
       2010          -2147483648  
       2011          -2147483648  
       2012          -2147483648  
...                          ...  
ZYXI   2015             -4071000  
       2016             -3802000  
       2017              4902000  
       2018              9291000  
       2019             19653000  

[18190 rows x 3 columns]

In [96]:
balance_sheet_tanglibles_gold.reset_index(inplace=True)

In [100]:
bal_sheet_tangibles_set = balance_sheet_tanglibles_gold.set_index(["Ticker"])
bal_sheet_tangibles_set

,Fiscal Year,"Property, Plant & Equipment, Net",Retained Earnings,Total Equity
Ticker,,,,
A,2008,824000000,-2147483648,-2147483648
A,2009,845000000,-2147483648,-2147483648
A,2010,980000000,-2147483648,-2147483648
A,2011,1006000000,-2147483648,-2147483648
A,2012,1164000000,-2147483648,-2147483648
...,...,...,...,...
ZYXI,2015,801000,-9845000,-4071000
ZYXI,2016,580000,-9776000,-3802000
ZYXI,2017,188000,-2411000,4902000


In [45]:
balance_sheet_intanglibles = balance_data[
    ['Treasury Stock', 
     'Long Term Investments & Receivables', 
     'Shares (Basic)', 
     'Shares (Diluted)']]

balance_intan_gold = balance_sheet_intanglibles.dropna().astype(int)

In [46]:
balance_intan_gold.reset_index(inplace=True)


In [101]:
balance_sheet_intangible_set = balance_intan_gold.set_index(['Ticker'])
balance_sheet_intangible_set

,Fiscal Year,Treasury Stock,Long Term Investments & Receivables,Shares (Basic),Shares (Diluted)
Ticker,,,,,
A,2008,-2147483648,206000000,363000000,371000000
A,2009,-2147483648,163000000,346000000,346000000
A,2010,-2147483648,142000000,347000000,353000000
A,2011,-2147483648,117000000,347000000,355000000
A,2012,-2147483648,109000000,348000000,353000000
...,...,...,...,...,...
ZEN,2014,-652000,9205000,53571000,53571000
ZEN,2015,-652000,22336000,84926000,84926000
ZEN,2016,-652000,75168000,93161000,93161000


In [58]:
balance_data_neg = balance_data[
    ["Inventories",
     "Share Capital & Additional Paid-In Capital", 
     "Payables & Accruals", "Total Noncurrent Liabilities", 
     "Long Term Debt", 
     'Total Liabilities & Equity', 
     'Total Current Liabilities']]

balance_sheet_neg_gold = balance_data_neg.dropna().astype(int)

In [59]:
balance_sheet_neg_gold.reset_index(inplace=True)

In [106]:
balance_sheet_neg_set = balance_sheet_neg_gold.set_index(['Ticker'])
balance_sheet_neg_set

,Fiscal Year,Inventories,Share Capital & Additional Paid-In Capital,Payables & Accruals,Total Noncurrent Liabilities,Long Term Debt,Total Liabilities & Equity,Total Current Liabilities
Ticker,,,,,,,,
A,2008,646000000,-2147483648,436000000,-2147483648,2125000000,-1582934592,1330000000
A,2009,552000000,-2147483648,307000000,-2147483648,-2147483648,-977934592,1123000000
A,2010,716000000,-2147483648,499000000,-2147483648,-2147483648,1106065408,-2147483648
A,2011,898000000,-2147483648,472000000,-2147483648,1932000000,467065408,1837000000
A,2012,1014000000,-2147483648,461000000,-2147483648,2112000000,1946065408,1893000000
...,...,...,...,...,...,...,...,...
ZYXI,2013,5002000,5617000,3765000,2689000,150000,17221000,9684000
ZYXI,2014,1935000,5733000,3425000,324000,311000,7114000,8057000
ZYXI,2015,305000,5863000,3339000,228000,216000,3696000,7539000


# us_cashflow_annual_df

In [26]:
us_cashflow_annual_df = pd.read_csv(
    Path('us-cashflow-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_cashflow_annual_df.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,45846,USD,2008,FY,2008-10-31,2008-12-19,2010-12-20,363000000.0,371000000.0,...,756000000,-140000000.0,150000000.0,-1.720000e+08,-3.850000e+08,NaN,1.600000e+07,-790000000.0,-7.880000e+08,-421000000
1,A,45846,USD,2009,FY,2009-10-31,2009-12-21,2011-12-16,346000000.0,346000000.0,...,408000000,-127000000.0,94000000.0,4.300000e+07,-4.000000e+06,NaN,7.480000e+08,-86000000.0,6.470000e+08,1074000000
2,A,45846,USD,2010,FY,2010-10-31,2010-12-20,2012-12-20,347000000.0,353000000.0,...,718000000,-114000000.0,38000000.0,-1.108000e+09,-1.174000e+09,0.0,7.180000e+08,-112000000.0,6.010000e+08,170000000
3,A,45846,USD,2011,FY,2011-10-31,2011-12-16,2013-12-19,347000000.0,355000000.0,...,1260000000,-170000000.0,16000000.0,-9.700000e+07,1.294000e+09,0.0,-1.500000e+09,-193000000.0,-1.693000e+09,878000000
4,A,45846,USD,2012,FY,2012-10-31,2012-12-20,2014-12-22,348000000.0,353000000.0,...,1228000000,-194000000.0,5000000.0,-2.257000e+09,-2.366000e+09,-104000000.0,1.500000e+08,-72000000.0,-3.500000e+07,-1174000000


In [27]:
dirty_cash_data = us_cashflow_annual_df.drop(['SimFinId', 'Currency'], axis=1)
cash_flow = dirty_cash_data.groupby(['Ticker','Fiscal Year']).mean()
cash_flow

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Net Income/Starting Line  Depreciation & Amortization  \
Ticker Fiscal Year                                                          
A      2008                     6.930000e+08                  201000000.0   
       2009                    -3.100000e+07                  162000000.0   
       2010                     6.840000e+08                  202000000.0   
       2011                     1.012000e+09                  253000000.0   
       2012                     1.153000e+09                  301000000.0   
...                                      ...                          ...   
ZYXI   2015                    -2.934000e+06                     424000.0   
       2016                     6.900000e+04                     435000.0   
       2017                     7.365000e+06                     286000.0   
       2018                     9.552000e+06                     448000.0   
       2019                     9.492000e+06                     778000.0   

                    Non-Cash Items  Change in Working Capital  \
Ticker Fiscal Year                                              
A      2008             41000000.0               -179000000.0   
       2009            215000000.0                 62000000.0   
       2010           -116000000.0                -52000000.0   
       2011            219000000.0               -224000000.0   
       2012            -50000000.0               -176000000.0   
...                            ...                        ...   
ZYXI   2015              1198000.0                  1653000.0   
       2016               192000.0                  1073000.0   
       2017              1771000.0                 -1162000.0   
       2018                15000.0                  -607000.0   
       2019               152000.0                 -4119000.0   

                    Change in Accounts Receivable  Change in Inventories  \
Ticker Fiscal Year                                                         
A      2008                           -44000000.0            -14000000.0   
       2009                           193000000.0             47000000.0   
       2010                          -166000000.0            -51000000.0   
       2011                            11000000.0           -208000000.0   
       2012                            19000000.0            -52000000.0   
...                                           ...                    ...   
ZYXI   2015                                   NaN                    NaN   
       2016                                   NaN                    NaN   
       2017                                   NaN                    NaN   
       2018                                   NaN                    NaN   
       2019                                   NaN                    NaN   

                    Change in Accounts Payable  Change in Other  \
Ticker Fiscal Year                                                
A      2008                        -21000000.0     -100000000.0   
       2009                         -7000000.0     -171000000.0   
       2010                        113000000.0       52000000.0   
       2011                        -35000000.0        8000000.0   
       2012                        -31000000.0     -

In [108]:
cash_flow_var = cash_flow[
    ["Change in Working Capital",
     "Change in Accounts Receivable",
     "Change in Inventories", 
     "Change in Accounts Payable"]]

cash_flow_change_gold = cash_flow_var.dropna().astype(int)

In [109]:
cash_flow_change_gold.reset_index(inplace=True)

In [110]:
cash_flow_change_set = cash_flow_change_gold.set_index(['Ticker'])
cash_flow_change_set

,Fiscal Year,Change in Working Capital,Change in Accounts Receivable,Change in Inventories,Change in Accounts Payable
Ticker,,,,,
A,2008,-179000000,-44000000,-14000000,-21000000
A,2009,62000000,193000000,47000000,-7000000
A,2010,-52000000,-166000000,-51000000,113000000
A,2011,-224000000,11000000,-208000000,-35000000
A,2012,-176000000,19000000,-52000000,-31000000
...,...,...,...,...,...
XYL,2015,15000000,-24000000,23000000,20000000
XYL,2016,61000000,-6000000,-15000000,61000000
XYL,2017,141000000,-79000000,27000000,50000000


In [84]:
cash_flow_positive = cash_flow[
    ['Net Cash from Operating Activities',
     'Net Cash from Acquisitions & Divestitures', 
     'Net Cash from Investing Activities', 
     'Cash from (Repayment of) Debt']]

pos_cash_flow_gold = cash_flow_positive.dropna().astype(int)

In [85]:
pos_cash_flow_gold.reset_index(inplace=True)                                                           

In [111]:
cash_flow_pos = pos_cash_flow_gold.set_index(['Ticker'])

In [112]:
cash_flow_pos

,Fiscal Year,Net Cash from Operating Activities,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Cash from (Repayment of) Debt
Ticker,,,,,
A,2008,756000000,-172000000,-385000000,16000000
A,2009,408000000,43000000,-4000000,748000000
A,2010,718000000,-1108000000,-1174000000,718000000
A,2011,1260000000,-97000000,1294000000,-1500000000
A,2012,1228000000,-2147483648,-2147483648,150000000
...,...,...,...,...,...
ZUMZ,2016,48455000,-5395000,-51515000,650000
ZUO,2017,-24776000,-11420000,-16118000,11607000
ZUO,2018,-23581000,-247000,-121123000,-3176000


In [70]:
cash_flow_negative = cash_flow[
    ["Dividends Paid", 
     "Cash from (Repurchase of) Equity", 
     "Net Cash from Financing Activities"]]

neg_cashflow_gold =cash_flow_negative.dropna().astype(int)

In [71]:
neg_cashflow_gold.reset_index(inplace=True)

In [113]:
neg_cash_flow_set = neg_cashflow_gold.set_index(["Ticker"])
neg_cash_flow_set

,Fiscal Year,Dividends Paid,Cash from (Repurchase of) Equity,Net Cash from Financing Activities
Ticker,,,,
A,2010,0,-112000000,601000000
A,2011,0,-193000000,-1693000000
A,2012,-104000000,-72000000,-35000000
A,2013,-156000000,-739000000,-554000000
A,2014,-176000000,-12000000,-117000000
...,...,...,...,...
ZTS,2016,-188000000,-300000000,-903000000
ZTS,2017,-206000000,-500000000,-251000000
ZTS,2018,-243000000,-698000000,533000000


# income-annual

In [115]:
us_income_annual_df = pd.read_csv(
    Path('us-income-annual.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_income_annual_df.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
0,A,45846,USD,2008,FY,2008-10-31,2008-12-19,2010-12-20,363000000.0,371000000.0,...,20000000.0,-10000000.0,815000000,0.0,815000000,-122000000.0,693000000,NaN,693000000,693000000
1,A,45846,USD,2009,FY,2009-10-31,2009-12-21,2011-12-16,346000000.0,346000000.0,...,-40000000.0,-59000000.0,7000000,0.0,7000000,-38000000.0,-31000000,NaN,-31000000,-31000000
2,A,45846,USD,2010,FY,2010-10-31,2010-12-20,2012-12-20,347000000.0,353000000.0,...,-6000000.0,-76000000.0,560000000,132000000.0,692000000,-8000000.0,684000000,NaN,684000000,684000000
3,A,45846,USD,2011,FY,2011-10-31,2011-12-16,2013-12-19,347000000.0,355000000.0,...,-39000000.0,-72000000.0,1032000000,NaN,1032000000,-20000000.0,1012000000,NaN,1012000000,1012000000
4,A,45846,USD,2012,FY,2012-10-31,2012-12-20,2014-12-22,348000000.0,353000000.0,...,-76000000.0,-92000000.0,1043000000,NaN,1043000000,110000000.0,1153000000,NaN,1153000000,1153000000


In [116]:
dirty_income = us_income_annual_df.drop(['SimFinId'], axis=1)


In [117]:
income_annual = dirty_income.groupby(['Ticker','Fiscal Year']).mean()

income_annual

Shares (Basic)  Shares (Diluted)       Revenue  \
Ticker Fiscal Year                                                   
A      2008            363000000.0       371000000.0  5.774000e+09   
       2009            346000000.0       346000000.0  4.481000e+09   
       2010            347000000.0       353000000.0  5.444000e+09   
       2011            347000000.0       355000000.0  6.615000e+09   
       2012            348000000.0       353000000.0  6.858000e+09   
...                            ...               ...           ...   
ZYXI   2015             31271234.0        31271234.0  1.164100e+07   
       2016             31271234.0        31271234.0  1.331300e+07   
       2017             32156000.0        33196000.0  2.343200e+07   
       2018             32503000.0        34043000.0  3.191700e+07   
       2019             32439000.0        33963000.0  4.547200e+07   

                    Cost of Revenue  Gross Profit  Operating Expenses  \
Ticker Fiscal Year                                                      
A      2008           -2.578000e+09  3.196000e+09       -2.401000e+09   
       2009           -2.189000e+09  2.292000e+09       -2.245000e+09   
       2010           -2.514000e+09  2.930000e+09       -2.364000e+09   
       2011           -3.086000e+09  3.529000e+09       -2.458000e+09   
       2012           -3.254000e+09  3.604000e+09       -2.485000e+09   
...                             ...           ...                 ...   
ZYXI   2015           -4.937000e+06  6.704000e+06       -9.185000e+06   
       2016           -3.517000e+06  9.796000e+06       -9.156000e+06   
       2017           -4.819000e+06  1.861300e+07       -9.669000e+06   
       2018           -6.038000e+06  2.587900e+07       -1.550900e+07   
       2019           -8.814000e+06  3.665800e+07       -2.559200e+07   

                    Selling, General & Administrative  Research & Development  \
Ticker Fiscal Year                                                              
A      2008                             -1.697000e+09            -704000000.0   
       2009                             -1.603000e+09            -642000000.0   
       2010                             -1.752000e+09            -612000000.0   
       2011                             -1.809000e+09            -649000000.0   
       2012                             -1.817000e+09            -668000000.0   
...                                               ...                     ...   
ZYXI   2015                             -9.185000e+06                     NaN   
       2016                             -9.156000e+06                     NaN   
       2017                             -9.669000e+06                     NaN   
       2018                             -1.550900e+07                     NaN   
       2019                             -2.559200e+07                     NaN   

                    Depreciation & Amortization  Operating Income (Loss)  \
Ticker Fiscal Year                                                         
A      2008                                 NaN                795000000   
       2009                                 NaN                 47000000   
       2010                                 NaN                566000000   
       2011                                 NaN               1071000000   
       2012                                 NaN               1119000000   
...                                         ...                      ...   
ZYXI   2015                                 NaN                 -2481000   
       2016                                 NaN                   640000   
       2017                                 NaN                  8944000   
       2018                                 NaN                 10370000   
       2019                                 NaN                 11066000   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Fiscal Year                        

In [ ]:
assess_NA(income_annual)

In [118]:
income_pos = income_annual[
    ["Gross Profit", 
     "Operating Income (Loss)", 
     "Income (Loss) from Continuing Operations", 
     "Net Income", 
     "Net Income (Common)"]]

income_pos_gold = income_pos.dropna().astype(int)

In [ ]:
income_pos_gold.reset_index(inplace=True)

In [122]:
income_pos_set = income_pos_gold.set_index(['Ticker'])
income_pos_set

,Fiscal Year,Gross Profit,Operating Income (Loss),Income (Loss) from Continuing Operations,Net Income,Net Income (Common)
Ticker,,,,,,
A,2008,-2147483648,795000000,693000000,693000000,693000000
A,2009,-2147483648,47000000,-31000000,-31000000,-31000000
A,2010,-2147483648,566000000,684000000,684000000,684000000
A,2011,-2147483648,1071000000,1012000000,1012000000,1012000000
A,2012,-2147483648,1119000000,1153000000,1153000000,1153000000
...,...,...,...,...,...,...
ZYXI,2015,6704000,-2481000,-2934000,-2911000,-2911000
ZYXI,2016,9796000,640000,69000,69000,69000
ZYXI,2017,18613000,8944000,7365000,7365000,7365000


In [124]:
in_income = income_annual[
    ["Net Extraordinary Gains (Losses)", 
     "Research & Development", 
     "Non-Operating Income (Loss)"]]

in_income_gold = in_income.dropna().astype(int)

In [125]:
in_income_gold.reset_index(inplace=True)

In [128]:
in_income_set = in_income_gold.set_index(["Ticker"])
in_income_set

,Fiscal Year,Net Extraordinary Gains (Losses),Research & Development,Non-Operating Income (Loss)
Ticker,,,,
A,2013,509000000,-337000000,-93000000
A,2014,317000000,-358000000,-190000000
A,2015,-37000000,-330000000,-42000000
ABAX,2012,-3668000,-13577000,253000
ABAX,2013,-2044000,-13647000,994000
...,...,...,...,...
ZGNX,2015,67848000,-27860000,-3030000
ZGNX,2016,-1021000,-41840000,-2336000
ZGNX,2017,-795000,-67449000,-1507000


In [130]:
income_neg = income_annual[
    ["Cost of Revenue", 
     "Operating Expenses", 
     "Selling, General & Administrative"]]

income_neg_gold = income_neg.dropna().astype(int)

In [131]:
income_neg_gold.reset_index(inplace=True)

In [133]:
income_neg_set = income_neg_gold.set_index(['Ticker'])
income_neg_set

,Fiscal Year,Cost of Revenue,Operating Expenses,"Selling, General & Administrative"
Ticker,,,,
A,2008,-2147483648,-2147483648,-1697000000
A,2009,-2147483648,-2147483648,-1603000000
A,2010,-2147483648,-2147483648,-1752000000
A,2011,-2147483648,-2147483648,-1809000000
A,2012,-2147483648,-2147483648,-1817000000
...,...,...,...,...
ZYXI,2015,-4937000,-9185000,-9185000
ZYXI,2016,-3517000,-9156000,-9156000
ZYXI,2017,-4819000,-9669000,-9669000


In [ ]:
income_annual.info()

# shareprices

In [134]:


us_shareprice_df = pd.read_csv(
    Path('us-shareprices-daily.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_shareprice_df.head()

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,GOOG,18,2014-03-27,568.00,552.92,568.00,558.46,558.46,NaN,13100,336050831.0
1,GOOG,18,2014-03-28,561.20,558.67,566.43,559.99,559.99,NaN,41100,336050831.0
2,GOOG,18,2014-03-31,566.89,556.93,567.00,556.97,556.97,NaN,10800,674462000.0
3,GOOG,18,2014-04-01,558.71,558.71,568.45,567.16,567.16,NaN,7900,674462000.0
4,GOOG,18,2014-04-02,565.11,562.19,604.83,567.00,567.00,NaN,146700,674462000.0


In [135]:
ticker_data = us_shareprice_df.groupby(['Ticker', 'Date']).mean()


In [138]:
ticker_data.reset_index(inplace=True)

In [140]:
share_price_set = ticker_data.set_index(['Ticker'])
share_price_set

,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
Ticker,,,,,,,,,,
A,2007-01-03,45846,34.99,34.05,35.48,34.30,22.72,NaN,2574600,NaN
A,2007-01-04,45846,34.30,33.46,34.60,34.41,22.80,NaN,2073700,NaN
A,2007-01-05,45846,34.30,34.00,34.40,34.09,22.58,NaN,2676600,NaN
A,2007-01-08,45846,33.98,33.68,34.08,33.97,22.51,NaN,1557200,NaN
A,2007-01-09,45846,34.08,33.63,34.32,34.01,22.53,NaN,1386200,NaN
...,...,...,...,...,...,...,...,...,...,...
ZYXI,2020-02-11,171401,10.02,9.55,10.02,9.88,9.88,NaN,196899,32791665.0
ZYXI,2020-02-12,171401,9.92,9.92,10.50,10.32,10.32,NaN,261188,32791665.0
ZYXI,2020-02-13,171401,10.24,10.10,10.36,10.26,10.26,NaN,120219,32791665.0


In [146]:
share_price_set["Close"]

Ticker
A       34.30
A       34.41
A       34.09
A       33.97
A       34.01
        ...  
ZYXI     9.88
ZYXI    10.32
ZYXI    10.26
ZYXI    10.16
ZYXI    10.18
Name: Close, Length: 5982676, dtype: float64

In [ ]:
ticktock_ticker

In [ ]:
good_list = [balance_data_cash_gold,
             income_pos_gold,
             cash_flow_positive,
             balane_sheet_tanglible_gold]
    

In [ ]:
good_list

In [ ]:
bad_list = [balance_data_neg,
            income_neg,
            cash_flow_negative
           ]

In [ ]:
bad_list

In [ ]:
best_list = [balane_sheet_tanglibles, in_income, cash_flow_var
    
]

In [ ]:
best_list